자연어 생성 모델
---
https://wikidocs.net/45101 참조한 예제

### RNN

데이터 전처리

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [1]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
# corpus 단어 토큰화
t = Tokenizer()

#전체 corpus 빈도기반 tokenizing 수행 -> 빈도가 높을수록 높은 번호를 부여
t.fit_on_texts([text])

# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [6]:
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [17]:
# 학습데이터 생성

sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0] #원래 문장을 토큰화된 수치로 치환
    print(encoded)
    for i in range(1, len(encoded)): #원래 문장을 찢어 학습데이터로 생성
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

sequences

[2, 3, 1, 4, 5]
[]
[6, 1, 7]
[]
[8, 1, 9, 10, 1, 11]
[]
학습에 사용할 샘플의 개수: 11


[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
# 학습데이터 길이 일치화
# 길이가 일치해야 RNN에 넣을 수 있다
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [16]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [15]:
# data/label 분리
# 문장의 마지막 단어를 label로 설정
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

X,y

(array([[ 0,  0,  0,  0,  2],
        [ 0,  0,  0,  2,  3],
        [ 0,  0,  2,  3,  1],
        [ 0,  2,  3,  1,  4],
        [ 0,  0,  0,  0,  6],
        [ 0,  0,  0,  6,  1],
        [ 0,  0,  0,  0,  8],
        [ 0,  0,  0,  8,  1],
        [ 0,  0,  8,  1,  9],
        [ 0,  8,  1,  9, 10],
        [ 8,  1,  9, 10,  1]]),
 array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11]))

In [18]:
# Label One-Hot Encoding 
# 딥러닝 네트워크는 오직 벡터 형태로만 출력을 해준다
# 단일값으로만 된 라벨을 one-hot encoding 해줘야 딥러닝 출력과 비교 가능
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

모델 설계

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [21]:
# 모델 생성방식은 CNN과 동일 - Sequential 방식
# 추후 tensorflow를 이용해 모델 내부를 뜯어보면 매우 복잡해질 수 있음
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(X, y, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 0s - loss: 2.4614 - acc: 0.3636
Epoch 2/200
11/11 - 0s - loss: 2.4486 - acc: 0.3636
Epoch 3/200
11/11 - 0s - loss: 2.4354 - acc: 0.4545
Epoch 4/200
11/11 - 0s - loss: 2.4219 - acc: 0.4545
Epoch 5/200
11/11 - 0s - loss: 2.4080 - acc: 0.4545
Epoch 6/200
11/11 - 0s - loss: 2.3936 - acc: 0.4545
Epoch 7/200
11/11 - 0s - loss: 2.3787 - acc: 0.4545
Epoch 8/200
11/11 - 0s - loss: 2.3632 - acc: 0.4545
Epoch 9/200
11/11 - 0s - loss: 2.3471 - acc: 0.4545
Epoch 10/200
11/11 - 0s - loss: 2.3303 - acc: 0.4545
Epoch 11/200
11/11 - 0s - loss: 2.3128 - acc: 0.4545
Epoch 12/200
11/11 - 0s - loss: 2.2945 - acc: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.2754 - acc: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2555 - acc: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2348 - acc: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.2132 - acc: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.1909 - acc: 0.3636
Epoch 18/200
11/11 - 0s - loss: 2.1678 - acc: 0.3636
Epoch 19/200
11/11 - 0s - loss: 2.1

Epoch 155/200
11/11 - 0s - loss: 0.2477 - acc: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2437 - acc: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2398 - acc: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2360 - acc: 1.0000
Epoch 159/200
11/11 - 0s - loss: 0.2322 - acc: 1.0000
Epoch 160/200
11/11 - 0s - loss: 0.2285 - acc: 1.0000
Epoch 161/200
11/11 - 0s - loss: 0.2248 - acc: 1.0000
Epoch 162/200
11/11 - 0s - loss: 0.2212 - acc: 1.0000
Epoch 163/200
11/11 - 0s - loss: 0.2176 - acc: 1.0000
Epoch 164/200
11/11 - 0s - loss: 0.2141 - acc: 1.0000
Epoch 165/200
11/11 - 0s - loss: 0.2107 - acc: 1.0000
Epoch 166/200
11/11 - 0s - loss: 0.2073 - acc: 1.0000
Epoch 167/200
11/11 - 0s - loss: 0.2040 - acc: 1.0000
Epoch 168/200
11/11 - 0s - loss: 0.2007 - acc: 1.0000
Epoch 169/200
11/11 - 0s - loss: 0.1975 - acc: 1.0000
Epoch 170/200
11/11 - 0s - loss: 0.1943 - acc: 1.0000
Epoch 171/200
11/11 - 0s - loss: 0.1912 - acc: 1.0000
Epoch 172/200
11/11 - 0s - loss: 0.1881 - acc: 1.0000
Epoch 173/200
11/11 - 0s - l

In [63]:
# 모델 출력 확인
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [62]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측
print(sentence_generation(model, t, '경마장에', 4))

ValueError: Error when checking input: expected embedding_2_input to have shape (23,) but got array with shape (5,)

In [25]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [26]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


In [28]:
# 생소한 단어에 대해서는 이상하게 동작하는것 확인 가능
print(sentence_generation(model, t, '싸피', 5))
print(sentence_generation(model, t, 'his',2))

싸피 말이 말이 고와야 오는 말이
his 말이 말이


### LSTM

데이터 준비

In [33]:
pwd

'E:\\ssafy\\s02p22a401\\doc\\RNN_summary'

In [30]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [34]:
# 뉴욕타임스 2018년 기사 데이터
df=pd.read_csv('nyt-comments/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [44]:
# Noise 제거
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [49]:
print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1214


In [47]:
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈값 제거 후 샘플의 개수 : 1214


In [50]:
#데이터 전처리 - 구두점,문장부호 제거, lower 적용
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [51]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [52]:
t.word_index #전치사, 관사 등의 문법단어는 그대로 살아있음 확인

{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'is': 8,
 'on': 9,
 'with': 10,
 'trump': 11,
 'as': 12,
 'at': 13,
 'new': 14,
 'how': 15,
 'from': 16,
 'it': 17,
 'an': 18,
 'that': 19,
 'be': 20,
 'season': 21,
 'us': 22,
 'you': 23,
 'its': 24,
 'what': 25,
 'episode': 26,
 'can': 27,
 'your': 28,
 'not': 29,
 'he': 30,
 'now': 31,
 'his': 32,
 'are': 33,
 'teaching': 34,
 'war': 35,
 'out': 36,
 'no': 37,
 'was': 38,
 'by': 39,
 'trumps': 40,
 'has': 41,
 'over': 42,
 'may': 43,
 'into': 44,
 'why': 45,
 'more': 46,
 'we': 47,
 'who': 48,
 'about': 49,
 'recap': 50,
 'activities': 51,
 '1': 52,
 'just': 53,
 'do': 54,
 'women': 55,
 'when': 56,
 'syria': 57,
 'trade': 58,
 'i': 59,
 '2': 60,
 'or': 61,
 'will': 62,
 'this': 63,
 'have': 64,
 'president': 65,
 'but': 66,
 'home': 67,
 'up': 68,
 'long': 69,
 'one': 70,
 'off': 71,
 'facebook': 72,
 'house': 73,
 'gop': 74,
 'our': 75,
 'case': 76,
 'they': 77,
 'life': 78,
 'end': 79,
 'right': 80,
 'some':

In [53]:
# 학습데이터 준비
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [54]:
# 인덱스 숫자와 단어 확인
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [55]:
# 샘플 길이 padding
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [56]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [57]:
# train/label 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [58]:
# label one-hot-encoding
y = to_categorical(y, num_classes=vocab_size)

모델 설계

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [60]:
#모델 생성
# 6s/epoch -> 
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7803 samples
Epoch 1/200
7803/7803 - 6s - loss: 7.6390 - acc: 0.0293
Epoch 2/200
7803/7803 - 6s - loss: 7.1206 - acc: 0.0306
Epoch 3/200
7803/7803 - 6s - loss: 6.9851 - acc: 0.0347
Epoch 4/200
7803/7803 - 6s - loss: 6.8553 - acc: 0.0417
Epoch 5/200
7803/7803 - 6s - loss: 6.7025 - acc: 0.0451
Epoch 6/200
7803/7803 - 6s - loss: 6.5293 - acc: 0.0482
Epoch 7/200
7803/7803 - 6s - loss: 6.3414 - acc: 0.0541
Epoch 8/200
7803/7803 - 6s - loss: 6.1462 - acc: 0.0555
Epoch 9/200
7803/7803 - 6s - loss: 5.9540 - acc: 0.0616
Epoch 10/200
7803/7803 - 6s - loss: 5.7675 - acc: 0.0673
Epoch 11/200
7803/7803 - 6s - loss: 5.5901 - acc: 0.0698
Epoch 12/200
7803/7803 - 6s - loss: 5.4226 - acc: 0.0754
Epoch 13/200
7803/7803 - 6s - loss: 5.2588 - acc: 0.0841
Epoch 14/200
7803/7803 - 6s - loss: 5.1049 - acc: 0.0887
Epoch 15/200
7803/7803 - 6s - loss: 4.9560 - acc: 0.0953
Epoch 16/200
7803/7803 - 6s - loss: 4.

Epoch 139/200
7803/7803 - 7s - loss: 0.3905 - acc: 0.9070
Epoch 140/200
7803/7803 - 6s - loss: 0.3693 - acc: 0.9111
Epoch 141/200
7803/7803 - 6s - loss: 0.3448 - acc: 0.9139
Epoch 142/200
7803/7803 - 6s - loss: 0.3337 - acc: 0.9154
Epoch 143/200
7803/7803 - 7s - loss: 0.3250 - acc: 0.9152
Epoch 144/200
7803/7803 - 6s - loss: 0.3228 - acc: 0.9154
Epoch 145/200
7803/7803 - 6s - loss: 0.3207 - acc: 0.9157
Epoch 146/200
7803/7803 - 6s - loss: 0.3244 - acc: 0.9167
Epoch 147/200
7803/7803 - 6s - loss: 0.3199 - acc: 0.9161
Epoch 148/200
7803/7803 - 6s - loss: 0.3167 - acc: 0.9148
Epoch 149/200
7803/7803 - 6s - loss: 0.3141 - acc: 0.9157
Epoch 150/200
7803/7803 - 7s - loss: 0.3127 - acc: 0.9166
Epoch 151/200
7803/7803 - 7s - loss: 0.3108 - acc: 0.9159
Epoch 152/200
7803/7803 - 6s - loss: 0.3103 - acc: 0.9171
Epoch 153/200
7803/7803 - 7s - loss: 0.3148 - acc: 0.9177
Epoch 154/200
7803/7803 - 7s - loss: 0.3099 - acc: 0.9164
Epoch 155/200
7803/7803 - 7s - loss: 0.3116 - acc: 0.9153
Epoch 156/200


In [87]:
reshape_X = X[0].reshape(1,23)

In [89]:
model.predict(reshape_X,verbose=0)

array([[1.8096646e-11, 1.4314705e-06, 2.1641354e-06, ..., 3.9318615e-09,
        1.8927975e-07, 8.9086463e-09]], dtype=float32)

In [88]:
model.predict_classes(reshape_X,verbose=0)

array([269], dtype=int64)

In [83]:
# 임의의 단어 이용 다음 단어 예측
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        print(encoded.shape)
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [84]:
print(sentence_generation(model, t, 'i', 1))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

(1, 23)
i disapprove


In [66]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how to make facebook more accountable hes beat trump but a
